# Thermal fin

## Problem statement

We consider the problem of designing a thermal fin to effectively remove heat from a surface. The two-dimensional fin, shown in Figure below, consists of a vertical central post and four horizontal subfins; the fin conducts heat from a prescribed uniform flux source at the root, $\Gamma_{\text {root }}$, through the large-surface-area subfins to surrounding flowing air. The fin is characterized by a five-component parameter vector, or input, $\mu_{=}\left(\mu_1, \mu_2, \ldots, \mu_5\right)$, where $\mu_i=k^i, i=1, \ldots, 4$, and $\mu_5=\mathrm{Bi} ; \mu$ may take on any value in a specified design set $D \subset \mathbb{R}^5$.

![ROOT:fin2d 4 mesh](ROOT:fin2d-4-mesh.png)

Here $k^i$ is the thermal conductivity of the ith subfin (normalized relative to the post conductivity $k^0 \equiv 1$ ); and $\mathrm{Bi}$ is the Biot number, a nondimensional heat transfer coefficient reflecting convective transport to the air at the fin surfaces (larger $\mathrm{Bi}$ means better heat transfer). For example, suppose we choose a thermal fin with $k^1=0.4, k^2=0.6, k^3=0.8, k^4=1.2$, and $\mathrm{Bi}=0.1$; for this particular configuration $\mu=\{0.4,0.6,0.8,1.2,0.1\}$, which corresponds to a single point in the set of all possible configurations D (the parameter or design set). The post is of width unity and height four; the subfins are of fixed thickness $t=0.25$ and length $L=2.5$.
We are interested in the design of this thermal fin, and we thus need to look at certain outputs or cost-functionals of the temperature as a function of $\mu$. We choose for our output $T_{\text {root }}$, the average steady-state temperature of the fin root normalized by the prescribed heat flux into the fin root. The particular output chosen relates directly to the cooling efficiency of the fin lower values of $T_{\text {root }}$ imply better thermal performance. The steadystate temperature distribution within the fin, $u(\mu)$, is governed by the elliptic partial differential equation

$$
\rho_i C_i \frac{\partial u^i}{\partial t} -k^i \Delta u^i=0 \text { in } \Omega^i, i=0, \ldots, 4,
$$
where $\Delta$ is the Laplacian operator, and $u_i$ refers to the restriction of $u$ to $\Omega^i$.
Here $\Omega^i$ is the region of the fin with conductivity $k^i, i=0, \ldots, 4$ and volumetric heat capacity $(\rho C)_i, i=0, \cdots, 4$: $\Omega^0$ is thus the central post, and $\Omega^i, i=1, \ldots, 4$, corresponds to the four subfins.
The entire fin domain is denoted $\Omega\left(\bar{\Omega}=\cup_{i=0}^4 \bar{\Omega}^i\right)$; the boundary $\Omega$ is denoted $\Gamma$. We must also ensure continuity of temperature and heat flux at the conductivity discontinuity interfaces $\Gamma_{\text {int }}^i \equiv \partial \Omega^0 \cap \partial \Omega^i, i=1, \ldots, 4$, where $\partial \Omega^i$ denotes the boundary of $\Omega^i$, we have on $\Gamma_{\text {int }}^i i=1, \ldots, 4$ :

$$
\begin{aligned}
u^0 & =u^i \\
-\left(\nabla u^0 \cdot n^i\right) & =-k^i\left(\nabla u^i \cdot n^i\right)
\end{aligned}
$$
here $n^i$ is the outward normal on $\partial \Omega^i$. Finally, we introduce a Neumann flux boundary condition on the fin root

$$
-\left(\nabla u^0 \cdot n^0\right)=-1 \text { on } \Gamma_{\text {root }},
$$
which models the heat source; and a Robin boundary condition

$$
-k^i\left(\nabla u^i \cdot n^i\right)=\operatorname{Bi} u^i \text { on } \Gamma_{e x t}^i, i=0, \ldots, 4,
$$
which models the convective heat losses. Here $\Gamma_{\text {ext }}^i$ is that part of the boundary of $\Omega^i$ exposed to the flowing fluid; note that $\cup_{i=0}^4 \Gamma_{e x t}^i=\Gamma \backslash \Gamma_{\text {root }}$. The average temperature at the root, $T_{\text {root }}(\mu)$, can then be expressed as $\ell^O(u(\mu))$, where

$$
\ell^O(v)=\int_{\Gamma_{\text {root }}} v
$$
## Implementation

First, we initialize the {feelpp} environment and set the working directory.


In [0]:
import feelpp
from feelpp_project import laplacian
import json
import os

d = os.getcwd()
print(f"directory={d}")
e = feelpp.Environment(['fin'], config=feelpp.localRepository("."))


Next, we set the configuration file for the simulation and load the specifications from a JSON file.


In [0]:
feelpp.Environment.setConfigFile(f"{d}/src/cases/laplacian/fin/fin1/fin2d.cfg")
# Reading the JSON file
data = laplacian.loadSpecs(f"{d}/src/cases/laplacian/fin/fin2d.json")
print(data)


Now, we create a Laplacian object, set the specifications, and run the simulation.


In [0]:
lap = laplacian.get(dim=2, order=1)
lap.setSpecs(data)
lap.run()
meas=lap.measures()


After running the simulation, we convert the results to a Pandas DataFrame and set the 'time' column as the index for easy data manipulation.


In [0]:
import pandas as pd
df = pd.DataFrame(meas)
df.set_index('time', inplace=True)
print(df.to_markdown())


In the next block, we plot the mean temperature values at the fin root and the exterior using Plotly.


In [0]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df["mean_Gamma_root"], mode='lines', name='T_{Gamma Root}'))
fig.add_trace(go.Scatter(x=df.index, y=df["mean_Gamma_ext"], mode='markers', name='T_{Gamma Ext}'))
fig.add_trace(go.Scatter(x=df.index, y=df["min"], mode='markers', name='min T'))
fig.add_trace(go.Scatter(x=df.index, y=df["max"], mode='markers', name='max T'))
fig.update_layout(title='Temperature', xaxis_title='time', yaxis_title='T')
fig.show()


Lastly, we plot the heat flux values at the fin root and the exterior.


In [0]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df.index, y=df["flux_Gamma_root"], mode='lines', name='Flux_{Gamma Root}'))
fig.add_trace(go.Scatter(
    x=df.index, y=df["flux_Gamma_ext"], mode='markers', name='Flux_{Gamma Ext}'))
fig.update_layout(title='Heat Flux', xaxis_title='time', yaxis_title='Flux')
fig.show()
